In [92]:
import tensorflow as tf
import librosa
import numpy as np
import IPython.display as ipd
import os
import pandas as pd
import pyaudio
import wave

## Loading samples

In [57]:
X = pd.read_csv(r"data\\samples.csv", compression='zip', index_col=0)
Y = pd.read_csv(r"data\\samples_y.csv", compression='zip', index_col=0)

In [603]:
X_val = X[:120].append(X[-120:], ignore_index=True)
X_train = X[120:-120]
Y_val = Y[:120].append(Y[-120:], ignore_index=True)
Y_train = Y[120:-120]

In [650]:
X_val

,1,2,3,4,5,6,7,8,9,10,...,2571,2572,2573,2574,2575,2576,2577,2578,2579,2580
0,-414.489990,-413.262604,-413.273346,-415.737518,-418.133148,-420.397186,-423.759888,-427.472290,-429.822083,-430.692322,...,6.080044,6.642385,6.777312,6.270924,5.917418,6.677174,8.254900,9.948101,10.823099,11.615138
1,-418.607300,-417.507660,-417.813782,-420.966339,-424.887177,-427.532623,-430.763611,-435.083405,-438.757080,-440.345734,...,7.070369,6.781847,5.422030,3.250249,1.345757,0.445168,0.480110,0.783154,1.446465,2.201390
2,-423.989227,-424.916443,-429.488464,-437.778503,-445.717560,-450.102905,-451.849609,-453.437836,-455.116943,-456.243256,...,5.947779,6.853555,7.333402,7.529510,7.480945,7.617231,7.987863,9.019009,10.254251,10.825352
3,-366.632477,-365.339325,-364.767670,-366.328613,-368.160431,-369.394226,-370.469147,-370.891876,-370.700256,-370.372803,...,8.545565,8.998903,8.682015,7.554102,7.093925,8.537359,10.689331,12.557087,13.762178,14.526439
4,-367.525574,-365.980713,-364.294220,-364.558777,-365.469025,-366.281555,-368.104919,-370.787781,-372.927216,-373.595947,...,9.855972,8.333395,5.809240,2.959773,0.924104,0.467078,2.532168,5.973138,8.704237,10.511824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,-297.021790,-293.326050,-290.262604,-291.854187,-294.049042,-292.671997,-290.731018,-291.324738,-290.488098,-272.019012,...,-4.670130,-3.044752,-2.924249,-3.604486,-4.217880,-4.206174,-2.685092,0.680055,2.239790,2.802963
236,-198.936310,-189.130341,-219.542328,-264.597687,-222.988739,-177.840546,-181.147766,-207.771179,-240.860489,-222.428131,...,11.241283,11.452251,19.700417,16.484730,16.949848,10.377247,9.851934,7.887486,-1.538523,-4.298707
237,-191.226639,-241.195099,-291.988434,-225.038498,-222.671127,-223.877457,-209.113510,-217.553375,-324.204102,-337.798981,...,-2.594204,-9.840267,-5.379720,-13.005196,-16.905325,-7.635345,-9.256220,-13.539808,-3.254010,-4.852815
238,-559.304016,-559.304016,-559.304016,-559.304016,-559.304016,-559.304016,-559.304016,-559.304016,-559.304016,-559.304016,...,-7.880740,-10.767847,-13.546814,6.125435,4.900060,1.128187,-3.437295,2.523351,-3.416727,-5.034337


## Preparing dataset

In [582]:
dataset = tf.data.Dataset.from_tensor_slices((X.values, Y.values))

In [583]:
shuffled_dataset = dataset.repeat().shuffle(len(X)).batch(128)

In [584]:
val_dataset = shuffled_dataset.take(700) 
train_dataset = shuffled_dataset.skip(700)

In [604]:
val_dataset = tf.data.Dataset.from_tensor_slices((X_val.values, Y_val.values)).repeat().shuffle(len(X_val)).batch(32)
train_dataset = tf.data.Dataset.from_tensor_slices((X_train.values, Y_train.values)).repeat().shuffle(len(X_train)).batch(256)

## Preparing model

In [635]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(20, activation='relu', input_shape=(X.shape[1],)),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(20, activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax'),
])

In [636]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## Train model

In [637]:
model.fit(train_dataset, epochs=1, validation_data=val_dataset, steps_per_epoch=14, validation_steps=8)

Train for 14 steps, validate for 8 steps
14/14 [==============================] - 1s 46ms/step - loss: 3.1961 - accuracy: 0.8926 - val_loss: 0.1900 - val_accuracy: 0.9492


## Obtaining answer

#### Recording sample

In [647]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 1.5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

* recording
* done recording


#### Sample processing

In [648]:
x, sr = librosa.load(WAVE_OUTPUT_FILENAME)
mfcc = librosa.feature.mfcc(y=x, sr=sr, hop_length=len(x)//128, n_mfcc=20)
mfcc = mfcc.flatten()
print(mfcc.shape)

ipd.Audio(WAVE_OUTPUT_FILENAME)

(2580,)


#### Model testing

In [649]:
# model.predict(X.values[2200].reshape(1, X.shape[1])
model.predict(mfcc.reshape(1, mfcc.shape[0]))

array([[9.9999976e-01, 2.4281019e-07]], dtype=float32)

#### Saving model

In [322]:
model.save('models\\128_hop_20_nmfcc(almost_almost_success).h5')